In [1]:
import sys, glob, os
from commons import convert_to_json, triq_optimization, qiskit_optimization, \
    calibration_type_enum, qiskit_compilation_enum, normalize_counts, calculate_success_rate_tvd, \
    convert_dict_binary_to_int, convert_dict_int_to_binary, sum_last_n_digits_dict
import wrappers.qiskit_wrapper as qiskit_wrapper
from wrappers.qiskit_wrapper import QiskitCircuit
import pandas as pd
import mthree
import mapomatic as mm
import mthree

from qiskit.quantum_info import SparsePauliOp
from qiskit import QuantumCircuit, transpile
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime.options import SamplerOptions, EstimatorOptions, DynamicalDecouplingOptions, TwirlingOptions

from qiskit_aer import AerSimulator, QasmSimulator, Aer
from qiskit.qasm2 import dumps
import matplotlib.pyplot as plt
import numpy as np

from qiskit.visualization import plot_histogram, plot_state_city
import qiskit.quantum_info as qi

import mitiq
from mitiq import zne, benchmarks

from qEmQUIP import QEM, conf

token = "476ea8c61cc54f36e4a21d70a8442f94203c9d87096eaad0886a3e8154d8c2e79bcad6f927c6050a76335dd68d783f478c1b828504748a4377b441c335c831aa"


In [2]:
q = QEM(runs=conf.runs, user_id=1, token=token, hw_name="ibm_brisbane")
qasm_files = q.get_qasm_files_from_path("./circuits/polar_sim/n3/x")
qasm_files = qasm_files
print(qasm_files)





ibm_brisbane
Connecting to quantum service...
Saving IBM Account...
Retrieving the real backend information of ibm_brisbane...
./circuits/polar_sim/n3/x
['./circuits/polar_sim/n3/x/polar_all_meas_n3_x.qasm']


In [3]:
q.set_backend(program_type="sampler", shots=1000)

In [4]:
noise_model, sim_noisy, coupling_map = qiskit_wrapper.get_noisy_simulator(q.backend, 2, False, method="statevector")

In [5]:
circ = QuantumCircuit.from_qasm_file(qasm_files[0])
circ.save_state()

pm = generate_preset_pass_manager(optimization_level=3, backend=sim_noisy)
transpiled_circuit = pm.run(circ)
transpiled_circuit.draw(idle_wires = False)



global phase: 5π/4
          ┌──────────┐┌────┐ ┌────────────┐                           »
q_4 -> 18 ┤ Rz(-π/2) ├┤ √X ├─┤ Rz(-1.666) ├───────────────────────────»
          ├──────────┤├────┤┌┴────────────┴┐┌────┐┌──────────┐┌──────┐»
q_5 -> 19 ┤ Rz(-π/2) ├┤ √X ├┤ Rz(-0.71594) ├┤ √X ├┤ Rz(-π/2) ├┤1     ├»
          ├─────────┬┘├────┤└┬────────────┬┘└────┘└──────────┘│  Ecr │»
q_3 -> 20 ┤ Rz(π/2) ├─┤ √X ├─┤ Rz(0.9099) ├───────────────────┤0     ├»
          ├─────────┴┐├────┤┌┴────────────┤                   └──────┘»
q_1 -> 21 ┤ Rz(-π/2) ├┤ √X ├┤ Rz(-2.1144) ├───────────────────────────»
          ├──────────┤├────┤├─────────────┤                   ┌──────┐»
q_2 -> 22 ┤ Rz(-π/2) ├┤ √X ├┤ Rz(-2.6015) ├───────────────────┤0     ├»
          ├──────────┤├────┤├─────────────┤ ┌────┐┌──────────┐│  Ecr │»
q_0 -> 23 ┤ Rz(-π/2) ├┤ √X ├┤ Rz(-2.0814) ├─┤ √X ├┤ Rz(-π/2) ├┤1     ├»
          └──────────┘└────┘└─────────────┘ └────┘└──────────┘└──────┘»
     c: 8/════════════════════════════════════════════════════════════»
                                                                      »
«                                                                     ┌──────┐»
«q_4 -> 18 ───────────────────────────────────────────────────────────┤0     ├»
«            ┌─────────┐  ┌────┐┌─────────────┐ ┌────┐   ┌──────────┐ │  Ecr │»
«q_5 -> 19 ──┤ Rz(π/2) ├──┤ √X ├┤ Rz(-1.7378) ├─┤ √X ├───┤ Rz(-π/2) ├─┤1     ├»
«          ┌─┴─────────┴─┐├────┤└─┬─────────┬─┘ └────┘   └──────────┘ └──────┘»
«q_3 -> 20 ┤ Rz(-0.6609) ├┤ √X ├──┤ Rz(π/2) ├─────────────────────────────────»
«          └─────────────┘└────┘  └─────────┘  ┌──────┐┌─────────────┐ ┌────┐ »
«q_1 -> 21 ────────────────────────────────────┤0     ├┤ Rz(-2.1144) ├─┤ √X ├─»
«           ┌────────────┐┌────┐  ┌──────────┐ │  Ecr │└─┬──────────┬┘ ├────┤ »
«q_2 -> 22 ─┤ Rz(2.3065) ├┤ √X ├──┤ Rz(-π/2) ├─┤1     ├──┤ Rz(-π/2) ├──┤ √X ├─»
«           └┬──────────┬┘├────┤ ┌┴──────────┴┐└┬────┬┘  ├─────────┬┘  └────┘ »
«q_0 -> 23 ──┤ Rz(-π/2) ├─┤ √X ├─┤ Rz(-2.631) ├─┤ √X ├───┤ Rz(π/2) ├──────────»
«            └──────────┘ └────┘ └────────────┘ └────┘   └─────────┘          »
«     c: 8/═══════════════════════════════════════════════════════════════════»
«                                                                             »
«          ┌───────────────┐                                         »
«q_4 -> 18 ┤ Rz(-0.095187) ├─────────────────────────────────────────»
«          └──┬─────────┬──┘ ┌────┐ ┌─────────────┐┌────┐┌──────────┐»
«q_5 -> 19 ───┤ Rz(π/2) ├────┤ √X ├─┤ Rz(-1.0218) ├┤ √X ├┤ Rz(-π/2) ├»
«             └─────────┘   ┌┴────┴┐└─┬─────────┬─┘├────┤└──────────┘»
«q_3 -> 20 ─────────────────┤1     ├──┤ Rz(π/2) ├──┤ √X ├────────────»
«              ┌────────┐   │  Ecr │  ├─────────┴┐ ├────┤ ┌────────┐ »
«q_1 -> 21 ────┤ Rz(-π) ├───┤0     ├──┤ Rz(-π/2) ├─┤ √X ├─┤ Rz(-π) ├─»
«           ┌──┴────────┴─┐ └┬────┬┘  ├─────────┬┘ └┬─┬─┘ └────────┘ »
«q_2 -> 22 ─┤ Rz(-1.7665) ├──┤ √X ├───┤ Rz(π/2) ├───┤M├──────|0>─────»
«           └─────────────┘  └────┘   └─────────┘   └╥┘              »
«q_0 -> 23 ──────────────────────────────────────────╫───────────────»
«                                                    ║               »
«     c: 8/══════════════════════════════════════════╩═══════════════»
«                                                    0               »
«                                                                        »
«q_4 -> 18 ──────────────────────────────────────────────────────────────»
«                      ┌─┐            ┌──────────┐   ┌────┐  ┌──────────┐»
«q_5 -> 19 ────────────┤M├────|0>─────┤ Rz(-π/2) ├───┤ √X ├──┤ Rz(-π/4) ├»
«            ┌──────┐  └╥┘┌──────────┐└──────────┘   └────┘  └─┬──────┬─┘»
«q_3 -> 20 ──┤1     ├───╫─┤ Rz(-π/2) ├─────────────────────────┤1     ├──»
«            │  Ecr │   ║ ├─────────┬┘   ┌────┐   ┌─────────┐  │  Ecr │  »
«q_1 -> 21 ──┤0     ├───╫─┤ Rz(π/2) ├────┤ √X ├───┤ Rz(π/2) ├──┤0     ├──»
«          ┌─┴──────┴─┐ ║ └──┬────┬─

In [14]:
result = sim_noisy.run(transpiled_circuit, shots = 1000).result()
noisy_statevector = result.get_statevector(transpiled_circuit)

sim_ideal = AerSimulator(method="statevector")
ideal_result = sim_ideal.run(transpiled_circuit, shots = 1000).result()
ideal_statevector = ideal_result.get_statevector(transpiled_circuit)

print("Noisy Simulator :", noisy_statevector.probabilities().round(5))

print("Ideal Simulator :", ideal_statevector.probabilities().round(5))



Noisy Simulator : [0.12452 0.      0.      0.      0.      0.12442 0.      0.      0.
 0.12557 0.      0.      0.12557 0.      0.      0.      0.      0.
 0.      0.      0.      0.      0.      0.      0.      0.      0.
 0.      0.      0.      0.      0.      0.      0.12558 0.      0.
 0.12558 0.      0.      0.      0.12443 0.      0.      0.      0.
 0.12433 0.      0.      0.      0.      0.      0.      0.      0.
 0.      0.      0.      0.      0.      0.      0.      0.      0.
 0.     ]
Ideal Simulator : [0.125 0.    0.    0.    0.    0.125 0.    0.    0.    0.125 0.    0.
 0.125 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.125 0.    0.
 0.125 0.    0.    0.    0.125 0.    0.    0.    0.    0.125 0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.   ]


In [15]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector

qc = QuantumCircuit(6)
qc.prepare_state(noisy_statevector, [0, 1, 2, 3, 4, 5])

qc.draw()

»
q_0: »
     »
q_1: »
     »
q_2: »
     »
q_3: »
     »
q_4: »
     »
q_5: »
     »
«     ┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
«q_0: ┤0                                                                                                                                                                                                                                                                                                                                                                                                                                                         ├
«     │                                                                                                                                                                                                                                                                                                                                                                                                                                                          │
«q_1: ┤1                                                                                                                                                                                                                                                                                                                                                                                                                                                         ├
«     │                                                                                                                                                                                                                                                                                                                                                                                                                                                          │
«q_2: ┤2                                                                                                                                                                                                                                                                                                                                                                                                                                                         ├
«     │  State Preparation(0.24952-0.24952j,0.00034539-0.00075472j,0,0,0.00034382-0.00075543j,0.24941-0.24942j,0,0,-0.00034508+0.00074964j,-0.25057+0.25057j,0,0,-0.25057+0.25057j,-0.00034343+0.0007487j,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.00078443+0.00031049j,-0.25058+0.25058j,0,0,-0.25058+0.25058j,-0.00078389+0.00030843j,0,0,0.24943-0.24943j,0.0007876-0.00031255j,0,0,0.0007873-0.00031196j,0.24933-0.24933j,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0) │
«q_3: ┤3                                                                                                                                                                                                                                                                                                                                                                                                                                                         ├
«     │                                                                                                                                                                                                                                                                                                           

In [ ]:
noisy_statevector.probabilities_dict()